In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

<h1>Disgenet adathalmaz feldolgozása</h1>


Disgenet esetében át kell váltani a kapcsolótábla segítségével a gén névről a gén azonosító használatára.

In [55]:
gen_name_gen_id = pd.read_csv("./data/gene_name_gene_id_map.txt", sep='\t', header=0)

#rename columns
gen_name_gen_id["gene_id"] = gen_name_gen_id["Gene stable ID"]
gen_name_gen_id["gene_name"] = gen_name_gen_id["Gene name"]
gen_name_gen_id = gen_name_gen_id[["gene_id", "gene_name"]]

gen_name_gen_id = gen_name_gen_id.drop_duplicates(subset=["gene_id"])
gen_name_gen_id

,gene_id,gene_name
0,ENSG00000261657,SLC25A26
1,ENSG00000223116,AL157931.1
2,ENSG00000233440,HMGA1P6
3,ENSG00000207157,RNY3P4
4,ENSG00000229483,LINC00362
...,...,...
66973,ENSG00000262334,RPH3AL
66974,ENSG00000262737,RP11-1260E13.1
66975,ENSG00000263267,RP11-1260E13.4
66976,ENSG00000262336,RP11-1260E13.2


In [56]:
disgenet = pd.read_csv("./data/disgenet_curated_gene_disease_associations.tsv", sep='\t', header=0)
disgenet

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,2,A2M,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84033,109580095,HBB-LCR,0.743,0.115,C0002875,Cooley's anemia,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84034,109580095,HBB-LCR,0.743,0.115,C0005283,beta Thalassemia,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84035,109580095,HBB-LCR,0.743,0.115,C0019025,Hemoglobin F Disease,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84036,109580095,HBB-LCR,0.743,0.115,C0085578,Thalassemia Minor,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human


A disgenetet érdemes, majd tovább szűrni az alapján, hogy hány ismert gén kapcsolata van.

In [57]:
#switch from gene name to gene id
disgenet_join = pd.merge(disgenet, gen_name_gen_id, left_on='geneSymbol', right_on='gene_name', how='inner')
disgenet_join = disgenet_join.drop_duplicates()
disgenet_result = disgenet_join.drop(columns=["geneSymbol", "gene_name", "geneId"])

#reorder the colums
df_col = disgenet_result.pop("gene_id")
disgenet_result.insert(0, "gene_id", df_col)
disgenet_result.to_csv("./data/raw/disgenet_with_gene_id.csv", sep="\t", index=False)
disgenet_result

,gene_id,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,ENSG00000121410,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,ENSG00000121410,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,ENSG00000175899,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,ENSG00000175899,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,ENSG00000175899,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92954,ENSG00000251908,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92955,ENSG00000222259,0.931,0.077,C2931456,"Prostate cancer, familial",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92956,ENSG00000222259,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human
92957,ENSG00000207222,0.931,0.077,C2931456,"Prostate cancer, familial",disease,C04;C12,Neoplastic Process,0.30,1.000,2018.0,2018.0,1,0,CTD_human


<h1>String adathalmaz feldolgozása</h1>

A string adathalmazból a legerősebb fehérje fehérje interakcióra van szükségünk. Ezek csak egyszer szerepeljenek a listában.

In [58]:
string = pd.read_csv("./data/string_9606.protein.links.v12.0.txt", sep=' ', header=0)

#remove the human prefix from gene ids
string["protein1"] = string["protein1"].str.slice(5)
string["protein2"] = string["protein2"].str.slice(5)

#normalize the combined score
string["combined_score"] = string["combined_score"] / 1000
string.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13715404 entries, 0 to 13715403
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   protein1        object 
 1   protein2        object 
 2   combined_score  float64
dtypes: float64(1), object(2)
memory usage: 313.9+ MB


Azon élek megtartása, melyek erősek.

In [59]:
string = string[string["combined_score"] >= 0.7]
string.shape

(473860, 3)

A string fehérja azonosítókat tartalmaz, ezeket összekell kötni a gén azonosítókkal.

In [60]:
gen_protein_map = pd.read_csv("./data/gene_protein_map.txt", sep='\t', header=0)
gen_protein_map = gen_protein_map.dropna()
gen_protein_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112012 entries, 5 to 170460
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Gene stable ID     112012 non-null  object
 1   Protein stable ID  112012 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [61]:
#join the genes with protein
string_gene_p1 = string.merge(gen_protein_map, left_on="protein1", right_on="Protein stable ID", how='left')
string_gene_p2 = string.merge(gen_protein_map, left_on="protein2", right_on="Protein stable ID", how='left')
string_gene_p1.shape

(473860, 5)

In [62]:
# keep from the joind tables only the gene id columns, and the combined score
string_gene = DataFrame()
string_gene["gene1"] = string_gene_p1["Gene stable ID"]
string_gene[["gene2", "combined_score"]] = string_gene_p2.loc[:,["Gene stable ID", "combined_score"]]
string_gene = string_gene.dropna()
string_gene.shape

(469024, 3)

In [63]:
string_sorted = string_gene.sort_values(by=["gene1", "combined_score"], ascending=[False, False])

#keep from gene-gene connection only the strongest one
string_filtered = string_sorted.drop_duplicates(subset=['gene1', 'gene2'], keep='first')
string_filtered = string_filtered.reset_index(drop=True)
string_filtered.shape

(468792, 3)

Mivel egy él két irányból is szerepelhet, ezeket kiszűröm. (Csak azért, hogy tisztán szerepeljenek az élek, később viszont szükségem lesz a duplán szerepléshez. De ezt akkor majd duplikálom)

In [64]:
#create a tuple to remove those connections which are like:  gene1, gene2 = gene2, gene1
string_filtered["tuple"] = string_filtered.apply(lambda row: tuple(sorted([row['gene1'], row['gene2']])), axis=1)
string_filtered = string_filtered.drop_duplicates(subset=['tuple'])
string_filtered = string_filtered.drop("tuple", axis=1)

string_filtered.to_csv("./data/raw/gene_graph.csv", index=False, sep="\t")

In [65]:
string_filtered.shape

(234397, 3)

<h1>GTEX adathalmaz feldolgozása</h1>

In [66]:
# collect the genes from the graph
g1 = string_filtered["gene1"]
g2 = string_filtered["gene2"]
gs = pd.DataFrame(pd.concat([g1, g2], ignore_index=True), columns=["genes"])
gs = gs.drop_duplicates()
gs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16109 entries, 0 to 468781
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genes   16109 non-null  object
dtypes: object(1)
memory usage: 251.7+ KB


In [67]:
gtex = pd.read_csv("./data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct", sep='\t', header=0)
gtex_sorted = gtex.sort_values(by="Name", ascending=True)

# remove non interessant .x endings from gene ids
gtex_sorted["Name"] = gtex_sorted.apply(lambda row: row["Name"].split(".")[0], axis=1)
gtex_sorted= gtex_sorted.reset_index(drop=True)
gtex_sorted

,Name,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,ENSG00000000003,TSPAN6,27.61980,24.00720,14.490600,9.333330,9.208420,7.393960,13.787200,6.189620,...,8.13486,8.84861,14.193900,7.775630,9.929470,65.609500,19.163000,28.292200,26.099300,0.071083
1,ENSG00000000005,TNMD,27.25540,12.27290,0.034486,0.132502,0.499037,0.699068,0.388347,0.110950,...,1.59104,3.41662,0.393012,0.153402,0.137227,0.184820,0.273954,0.379664,0.219229,0.000000
2,ENSG00000000419,DPM1,56.12930,53.30300,56.822000,64.709400,59.266800,63.760300,54.111700,17.159200,...,50.86900,49.23020,45.820000,55.823900,41.499000,78.189500,61.302100,70.791800,60.445300,12.483400
3,ENSG00000000457,SCYL3,7.19633,5.19482,5.183300,5.696390,6.190740,6.468040,8.823700,1.620270,...,7.67282,8.78579,6.405000,6.649510,5.297310,7.557520,9.584620,9.213180,8.623300,3.062680
4,ENSG00000000460,C1orf112,2.74343,1.85840,1.388130,1.971830,2.165610,2.358640,2.721160,0.538449,...,2.19681,2.38017,2.197030,2.565050,1.413090,10.709700,2.851260,4.001080,3.297850,0.657812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56195,ENSG00000284592,RP11-71G12.2,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56196,ENSG00000284594,MIR7847,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56197,ENSG00000284595,MIR6785,0.00000,0.00000,0.585770,0.000000,0.109876,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.426180,0.552425,0.000000,0.719890,0.596907,0.624359,0.000000,0.000000
56198,ENSG00000284596,MIR4467,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Azon gének megtalálása melyek szerepelnek a stringből készült gráfban és a gtex leíróban is. Ha egy génnek egy tulajdonságához nincsen értéke, akkor a többi gén azonos tulajdonságainak a mediánját használva kiegészítem.

In [68]:
gtex_mapped = pd.merge(gtex_sorted, gs, left_on='Name', right_on='genes', how='right')
gtex_mapped = gtex_mapped.drop(columns=["Name"])
cols = [col for col in gtex_mapped.columns if col != "genes" and col != "Description" ]
print(gtex_mapped.info())
gtex_mapped.loc[:, cols] = gtex_mapped.loc[:, cols].fillna(gtex_mapped.loc[:, cols].mean())

#move genes column to the first column
gtex_mapped = gtex_mapped[["genes"] + [col for col in gtex_mapped.columns if col != "genes"]]
gtex_mapped.to_csv("./data/raw/gtex_genes.csv", sep="\t", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16127 entries, 0 to 16126
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Description                                15898 non-null  object 
 1   Adipose - Subcutaneous                     15898 non-null  float64
 2   Adipose - Visceral (Omentum)               15898 non-null  float64
 3   Adrenal Gland                              15898 non-null  float64
 4   Artery - Aorta                             15898 non-null  float64
 5   Artery - Coronary                          15898 non-null  float64
 6   Artery - Tibial                            15898 non-null  float64
 7   Bladder                                    15898 non-null  float64
 8   Brain - Amygdala                           15898 non-null  float64
 9   Brain - Anterior cingulate cortex (BA24)   15898 non-null  float64
 10  Brain - Caudate (basal